In [1]:
from glob import glob
import pandas as pd
DATA_DIR = "/data/data/fintuning/wanjuan_exam/part-003756-a894b46e.jsonl"
from tqdm import tqdm
import json
import re
import random

In [3]:
def format_example(item, cot=False):
    example = item["q_main"] + "\n"
    choice_map = {"A":"option_a", "B":"option_b", "C":"option_c", "D":"option_d"} 
    for choice in ["A","B","C","D"]:
        if item[choice_map[choice]] != "":
            example += f'{choice}. {item[choice_map[choice]]}\n'
    # 非cot
    if not cot:
        answer = f'{item["std_ans"]}。{item["answer_detail"]}'
    # cot
    else:
        answer = f'{item["answer_detail"]}'
        pattern = r"故选.*"
        answer = re.sub(pattern, '', answer, flags=re.DOTALL)
        answer = answer + f'答案为{item["std_ans"]}'
    return example, answer

all_data = []
no_explain_counter = 0
cot = True
with open(DATA_DIR, 'r') as file:
    for idx, line in enumerate(file):
        # 解码JSON对象
        json_obj = json.loads(line)
        if json_obj["std_ans"]=="" or len(json_obj["answer_detail"])<10:
            # 200w数据这块continue
            choice, answer = format_example(json_obj, False)
            instruct = f"以下是一道单选题:\n{choice}请给出答案。\n"
            a_data = {
                    "instruction": instruct,
                    "input": "",
                    "output": answer
                }
            all_data.append(a_data)
        else:
            # 处理json_obj
            choice, answer = format_example(json_obj, cot)
            if not cot:
                instruct = f"以下是一道单选题:\n{choice}请给出答案。\n"
            # cot
            else:
                instruct = f"以下是一道{json_obj['q_type']}:\n{choice}请先给出解释再给出答案。\n"
            a_data = {
                    "instruction": instruct,
                    "input": "",
                    "output": answer
                }
            # print(a_data)
            all_data.append(a_data)



In [4]:
print(len(all_data))

3997183


In [5]:
print("filter before:", len(all_data))
filted_all_data = [x for x in all_data if "略" not in x["output"]]
print("filter after:", len(all_data))

filter before: 3997183
filter after: 3997183


In [4]:
print(filted_all_data[1000])

{'instruction': '以下是一道单选题:\n三位同学在一起讨论有关科技革命的话题，以下是他们谈话内容的片断：他们讨论这一话题的中心是（ ）  \n甲：科技革命成果的商品化周期只需要3～5年  \n乙：新技术之间联系密切，相互促进  \n丙：在经济增长的因素中，科技进步的因素占80%\nA. 第三次科技革命的影响\nB. 第三次科技革命的特点\nC. 第三次科技革命的成就\nD. 第三次科技革命的内容\n请给出答案。\n', 'input': '', 'output': 'B。根据所学知识，第三次科技革命与前两次科技革命相比，有下列特点：首先，这次科技革命不仅涌现了大量的科学成果，而且大大加快了科学技术转化为生产力的速度，缩短了知识变为物质财富的过程。其次，科学技术的各个领域之间相互渗透，一种技术的发展引起好几种技术的革命。第三，新技术成为社会生产力中最活跃的因素，在促进经济增长的各种因素中，科技进步所占的比重不断上升。甲、乙、丙同学的谈话都与第三次科技革命的特点有关。故选B  \n【点评】本题主要考查第三次科技革命。本题既考查了第三次科技革命的特点，也考查了学生提取有效信息的能力；学生应注意对三次科技革命的特点进行对比掌握。'}


In [5]:
import random
few_data = random.sample(filted_all_data, 200000)
print(len(few_data))
print(few_data[100])

200000
{'instruction': '以下是一道单选题:\n【题文】神经系统中，专门调节心跳、血压等重要生命活动的神经传导中枢在：\nA. 大脑皮层\nB. 小脑\nC. 脑干\nD. 脊髓\n请给出答案。\n', 'input': '', 'output': 'C。  \n试题分析：A、大脑由左右两个半球组成，大脑皮层的表面是灰质，称为大脑皮层，它是调节人体生理活动的最高级中枢，有躯体运动中枢、躯体感觉中枢、视觉中枢、听觉中枢等，B、小脑的主要功能是协调运动，维持身体的平衡，D、脊髓位于脊柱的椎管内，包括灰质和白质，具有反射和传导的功能，故A、B、D都不符合题意；  \nC、脑干中有能够调节人体基本生命活动的中枢，如呼吸中枢、心血管运动中枢等，一旦损伤会危及生命，所以脑干被称为“生命中枢”，故符合题意。  \n考点：本题考查的是神经系统的功能，解答此类题目的关键是熟记神经系统的功能。'}


In [6]:
import json
json_dir = "/data/data/fintuning/llamafactory_input/wanjuan_exam_390w.jsonl"
with open(json_dir, 'w', encoding="utf-8") as f:
    for item in filted_all_data:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

In [7]:
print(filted_all_data[0])

3837404
